<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg" align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">Trabajo Fin de Máster</p>
<p style="margin: 0; text-align:right;">Máster universitario en Ciencia de datos (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Autor: César Fernández Domínguez</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# Análisis de resultados del ajuste de hiperparámetros de cada uno de los modelos

Este notebook contiene el análisis de los resultados obtenidos en el notebook anterior, en el cual realizamos una parametrización de los distintos modelos propuestos para realizar la predicción de contaminantes del aire. 

Este análisis se ha organizado en los siguientes apartados:

 <ol start="1">
  <li>Carga de datos
  <li>Análisis de resultados
  <br>2.1. Resultados para la predicción del Óxido de Carbono ( $CO$ )
  <br>2.2. Resultados para la predicción del Óxido de Nitrógeno ( $NO_x$ )
  <br>2.3. Resultados para la predicción del Dióxido de Nitrógeno ( $NO_2$ )
  <br>2.4. Resultados para la predicción del Ozono ( $O_3$ )
</ol>
   
En primer lugar, cargamos algunas librerías que vamos a necesitar, y definimos la carpeta raiz de donde cargaremos los resultados del ajuste de hiperparámetros anterior:

In [2]:
# Mount folder in Google Drive (only for execution in Google Colab)
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
import sys, os
import numpy as np
import pandas as pd

rootDataFolder="../data/"
#rootDataFolder="/content/drive/My Drive/colab/data/"

# 1. Carga de datos

Cargamos los resultados de la ejecución del anterior notebook en un dataframe.

In [4]:
# Read tuning data file
filename = "tuning/tuning.csv"
tuning_data = pd.read_csv(os.path.join(rootDataFolder, filename), sep=',', encoding = "UTF-8")

Además, cargamos los metadatos para cada una de las estaciones de observación de calidad del aire en Barcelona. Esto nos será útil para la visualización de los resultados.

In [5]:
# Read air stations data file
data_air_stations = pd.read_csv(os.path.join(rootDataFolder, "data_air_stations.csv"), sep=',', encoding = "UTF-8")

# 2. Análisis de resultados

Inicialmente, construimos una función para mostrar una tabla de resultados para una estación y variable dependiente dada. En cada caso, se mostrará la mejor parametrización para cada modelo de predicción y método de selección de variables evaluados:

In [6]:
from IPython.display import display
import seaborn as sns;
   
def show_tuning_table(tuning_data, station, output, data_air_stations):
    '''
    Show a table with best results for each model and selection variables method for a station and output given.
    
    Inputs:
    
        - tuning_data: dataframe with global results of the model parameterization.
        - station: code of station.
        - output: column of dependant variable.
        - data_air_stations: air stations metadata
    '''
    
    # Select best parameter selection for each model and vars selection method w.r.t. a station and output given.
    data = tuning_data.loc[(tuning_data['CODI_EOI'] == station) & (tuning_data['y_columns'] == output)] \
         .groupby(['Model','vars_selection_method'], sort=True) \
         .apply(lambda x: x.sort_values(['RMSE']) \
         [['Model','vars_selection_method','x_columns','params','Duration','R2','RMSE']].head(1)) \
         .reset_index(drop=True)
    
    # Complete formatted name for each dependant variable 
    lut_pollutant = {"['CO']":"Carbon Oxide <b>( CO )</b>",
                     "['NOX']":"Nitrogen Oxide <b>( NO<sub>x</sub> )</b>",
                     "['NO2']":"Nitrogen Dioxide <b>( NO<sub>2</sub> )</b>",
                     "['O3']":"Ozone <b>( O<sub>3</sub> )</b>"}
    
    # Get name and type of air station
    nom_estacio=data_air_stations.loc[data_air_stations["CODI_EOI"] == station,'NOM_ESTACIO'].iloc[0]
    type_estacio=data_air_stations.loc[data_air_stations["CODI_EOI"] == station,'TIPUS_ESTACIO'].iloc[0]
    
    # Show table with results
    rcm = sns.light_palette('seagreen', as_cmap=True, reverse=True)
    cm = sns.light_palette('seagreen', as_cmap=True)

    title = '<b>{}</b> ( type: <b>{}</b> ) <br>{}'.format(nom_estacio, type_estacio, lut_pollutant[output])
    display(data.style.set_table_styles([{'selector': 'caption',
                           'props': [('font-size', '18px')]},
                                         {'selector': 'th',
                           'props': [('border', '2px solid black'),('text-align', 'center')]}])\
             #.set_table_attributes('class="pure-table"') \
             .set_properties(**{'max-width': '300px', 'border': '1px solid black', 'text-align':'center'}) \
             .set_caption(title).set_precision(2).hide_index() \
             .background_gradient(cmap=cm,subset=['R2']) \
             .background_gradient(cmap=rcm,subset=['RMSE','Duration']))
             #.highlight_min(subset=['RMSE','Duration'], color='green') \
             #.highlight_max(subset=['R2'], color='green'))

A continuación, iremos mostrando los resultados para cada una de las estaciones y contaminante predicho. En cada caso, incluiremos un análisis de los resultados. 

## 2.1. Resultados para la predicción del Óxido de Carbono ( $CO$ )

Empezamos analizando los resultados para los distintos modelos utilizados para predecir la concentración de **óxido de carbono** ( $CO$ ), en cada una de las estaciones de observación del aire. 

El óxido de carbono, o mónoxido de carbono, es un gas sin color ni olor emitido como consecuencia de la combustión incompleta de carburantes fósiles y de biocombustibles. En las ciudades, la principal fuente de generación de este gas esta en la combustión de los sistemas de calefacción. Este gas penetra en el organismo a través de los pulmones, y puede provocar una disminución de la capacidad de transporte de oxígeno de la sangre, con el consecuente detrimento de oxigenación de órganos y tejidos, así como disfunciones cardiacas, daños en el sistema nervioso, dolor de cabeza, mareos y fatiga; estos efectos pueden producirse tanto sobre el ser humano como sobre la fauna silvestre. Contribuye a la formación de gases de efecto invernadero: su vida media en la atmósfera es de unos tres meses, lo que permite su lenta oxidación para formar CO2, proceso durante el cual también se genera O3. [Hernández & Querol, 2013].

La primera estación analizada es la del **Eixample**, la cual está clasificada de tipo **traffic**. Para esta estación tenemos la siguiente tabla de mejores resultados:

In [7]:
show_tuning_table(tuning_data, 8019043, "['CO']", data_air_stations)

Model,vars_selection_method,x_columns,params,Duration,R2,RMSE
GRU,fs,"['NOX', 'NO2', 'HR']","{'batch_size': 10, 'epochs': 10, 'neurons': 100}",4.3,-0.14,0.14
GRU,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'VV10']","{'batch_size': 50, 'epochs': 50, 'neurons': 500}",16,-0.26,0.15
GRU,pcas,"['HR', 'NOX', 'P', 'PPT', 'RS', 'traffic']","{'batch_size': 50, 'epochs': 10, 'neurons': 500}",3.8,-0.61,0.16
GRU,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 10, 'epochs': 10, 'neurons': 100}",3.2,-0.054,0.13
LSTM,fs,"['NOX', 'PPT']","{'batch_size': 100, 'epochs': 10, 'neurons': 50}",2.6,-0.17,0.14
LSTM,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'VV10']","{'batch_size': 50, 'epochs': 50, 'neurons': 500}",13,-0.36,0.15
LSTM,pcas,"['HR', 'NOX', 'P', 'PPT', 'RS', 'traffic']","{'batch_size': 10, 'epochs': 100, 'neurons': 100}",14,-0.54,0.16
LSTM,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 10, 'epochs': 10, 'neurons': 500}",8.2,-0.25,0.15
MLP,fs,"['NOX', 'NO2', 'O3']","{'batch_size': 100, 'epochs': 10, 'neurons': 50}",0.91,-0.39,0.15
MLP,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'VV10']","{'batch_size': 50, 'epochs': 10, 'neurons': 500}",1.9,-0.14,0.14


En esta tabla, tenemos una primera columna con el nombre del modelo utilizado, una segunda columna con el método de selección de variables independinetes utilizado, la siguiente columna contiene la lista de variables independiente utilizada (en los supuestos de utilizar un método de reducción de variables, como PCA o FS, no siempre serán la misma selección de variables pues esto dependerá de la variabilidad y distribución de los datos en cada caso). La cuarta columna contiene la combinación de parámetros seleccionada como la mejor para cada modelo y método de selección de variables independientes. Las siguientes columnas, las cuales están marcadas con un gradiente de colores verdosos, indicando un color más intenso un mejor valor y un valor más claro uno peor, tenemos el tiempo de ejecución en cada ejecución del modelo, el valor de $R^2$ y el $RMSE$.

Por lo tanto, para esta estación y variable predicha, tenemos que los mejores resultados los obtenemos para los modelos basados en redes neuronales recurrentes, utilizando el total de variables independientes. Vemos que utilizando el método de selección de variables $FS$, la reducción de eficiencia de cada modelo no se ve reducida. También observamos como, del método $FS$, que la variable independiente que más influye en la predicción del contaminante $CO$, es el óxido de nitrógeno ( $NO_x$ ). Vemos como la influencia de los valores registrados de tránsito de vehículos tiene excasa influencia en los modelos.

A continuación, vemos los resultados para la estación de **Gracia - Sant Gervasi**:

In [8]:
show_tuning_table(tuning_data, 8019044, "['CO']", data_air_stations)

Model,vars_selection_method,x_columns,params,Duration,R2,RMSE
GRU,fs,"['NOX', 'NO2', 'NO']","{'batch_size': 50, 'epochs': 10, 'neurons': 100}",6.7,0.3,0.15
GRU,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'VV10']","{'batch_size': 10, 'epochs': 10, 'neurons': 50}",7.9,0.43,0.13
GRU,pcas,"['HR', 'NOX', 'SO2', 'T', 'traffic']","{'batch_size': 10, 'epochs': 100, 'neurons': 20}",17,0.29,0.15
GRU,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 50, 'epochs': 100, 'neurons': 100}",7.3,0.47,0.13
LSTM,fs,"['NOX', 'NO2', 'NO', 'HR']","{'batch_size': 100, 'epochs': 50, 'neurons': 20}",6.8,0.29,0.15
LSTM,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'VV10']","{'batch_size': 10, 'epochs': 10, 'neurons': 500}",13,0.43,0.13
LSTM,pcas,"['HR', 'NOX', 'SO2', 'T', 'traffic']","{'batch_size': 50, 'epochs': 100, 'neurons': 50}",6.7,0.29,0.15
LSTM,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 10, 'epochs': 10, 'neurons': 100}",5,0.44,0.13
MLP,fs,"['NOX', 'NO2', 'P']","{'batch_size': 100, 'epochs': 50, 'neurons': 20}",2.6,0.32,0.14
MLP,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'VV10']","{'batch_size': 10, 'epochs': 10, 'neurons': 50}",2.9,0.41,0.13


Se repiten los mejores resultados para los modelos basados en redes neuronales recurrentes. Así como la influencia del contaminante óxido de nitrógeno ( $NO_x$ ) en la predicción del valor de óxido de carbono ( $CO$ ). En este caso, sin embargo, se obtienen optimos resultados utilizando un método de selección de variables independientes basado en el primer componente de un análisis de PCA.

Seguidamente, se muestra los resultados para la estación de **Parc Vall Hebron**, clasificada como **background**:

In [9]:
show_tuning_table(tuning_data, 8019054, "['CO']", data_air_stations)

Model,vars_selection_method,x_columns,params,Duration,R2,RMSE
GRU,fs,"['NO', 'DV10', 'RS', 'NO2', 'P']","{'batch_size': 10, 'epochs': 10, 'neurons': 100}",6.8,0.16,0.086
GRU,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3']","{'batch_size': 100, 'epochs': 100, 'neurons': 50}",7.5,0.64,0.056
GRU,pcas,"['DV10', 'NOX', 'P', 'PPT', 'RS', 'SO2']","{'batch_size': 10, 'epochs': 10, 'neurons': 50}",3.7,0.18,0.085
GRU,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 50, 'epochs': 100, 'neurons': 20}",5.2,0.65,0.055
LSTM,fs,"['NO', 'DV10', 'P', 'NOX']","{'batch_size': 10, 'epochs': 100, 'neurons': 500}",65,-0.13,0.1
LSTM,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3']","{'batch_size': 50, 'epochs': 100, 'neurons': 50}",8.5,0.64,0.057
LSTM,pcas,"['DV10', 'NOX', 'P', 'PPT', 'RS', 'SO2']","{'batch_size': 10, 'epochs': 10, 'neurons': 500}",9.5,0.16,0.086
LSTM,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 100, 'epochs': 100, 'neurons': 20}",3.9,0.65,0.055
MLP,fs,"['NO', 'DV10', 'T', 'P', 'RS']","{'batch_size': 100, 'epochs': 50, 'neurons': 20}",2,0.14,0.087
MLP,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3']","{'batch_size': 100, 'epochs': 100, 'neurons': 20}",2.9,0.65,0.056


En este caso, se obtienen valores de $RMSE$ muy bajos, lo cual nos indica que los valores predichos se ajustandan en gran medida a los reales. Vemos que el método de selección de variables que mejor a funcionado, para esta estación, es el denominado $PCA0$. 

Por último, mostramos los resultados para la estación de **Palau Reial**, de tipo **background**:

In [10]:
show_tuning_table(tuning_data, 8019057, "['CO']", data_air_stations)

Model,vars_selection_method,x_columns,params,Duration,R2,RMSE
GRU,fs,"['NOX', 'NO2', 'RS', 'traffic', 'T']","{'batch_size': 100, 'epochs': 10, 'neurons': 500}",2.7,0.29,0.086
GRU,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3']","{'batch_size': 50, 'epochs': 10, 'neurons': 50}",5.3,0.36,0.082
GRU,pcas,"['HR', 'NOX', 'P', 'RS', 'traffic']","{'batch_size': 10, 'epochs': 10, 'neurons': 20}",2.5,0.26,0.088
GRU,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 100, 'epochs': 100, 'neurons': 20}",2.9,0.35,0.082
LSTM,fs,"['NOX', 'NO2', 'traffic']","{'batch_size': 100, 'epochs': 50, 'neurons': 50}",2,0.24,0.089
LSTM,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3']","{'batch_size': 50, 'epochs': 10, 'neurons': 500}",6.3,0.36,0.082
LSTM,pcas,"['HR', 'NOX', 'P', 'RS', 'traffic']","{'batch_size': 100, 'epochs': 50, 'neurons': 100}",2.3,0.26,0.088
LSTM,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 50, 'epochs': 50, 'neurons': 20}",2.4,0.37,0.081
MLP,fs,"['NOX', 'traffic', 'T']","{'batch_size': 10, 'epochs': 10, 'neurons': 100}",1.1,0.28,0.087
MLP,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3']","{'batch_size': 100, 'epochs': 10, 'neurons': 100}",2.5,0.32,0.084


Como vemos, los resultados son muy parecidos a los obtenidos en la anterior estación analizada de **Parc Vall Hebron**, del mismo tipo. 

En todas las estaciones hemos visto como, a parte de obtener los mejores resultados con los modelos *MLP*, *LSTM* y *GRU*, las variables independientes, o predictores, seleccionadas por ser aquellos que tienen mayor relevancia en el primer componente del *PCA*, nos permiten realizar las mejores predicciones de cada modelo. Además vemos que estas variables son todas relativas a contaminantes, observando la poca influencia que tiene las variables meteorológicas y de tránsito de vehículos, incluidas en este estudio, sobre la predicción del valor de óxido de carbono ( $CO$ )

## 2.2. Resultados para la predicción del Óxido de Nitrógeno ( $NO_x$ )

Analizamos ahora, en este apartado, los resultados obtenidos cuando tratamos de predecir los niveles de óxido de nitrógeno ( $NO_x$ ). 

Los denominados óxidos de nitrógeno engloban tanto al monóxido ( $NO$) como al dióxido de nitrógeno ( $NO_2$ ) ( $NO_x = NO + NO_2$ ). Como contaminantes, son gases que se emiten en los procesos de combustión que se llevan a cabo en relación con el tráfico (sobre todo vehículos automóviles, y en especial de motores diésel) y con el transporte en general, así como en instalaciones industriales de alta temperatura y de generación eléctrica. Producen importantes efectos adversos sobre la salud humana, el medio ambiente o, incluso, sobre las edificaciones. [Hernández & Querol, 2013].

Como antes, empezamos analizando los resultados obtenidos para la estación de observación del aire del **Eixample**.

In [11]:
show_tuning_table(tuning_data, 8019043, "['NOX']", data_air_stations)

Model,vars_selection_method,x_columns,params,Duration,R2,RMSE
GRU,fs,"['NO', 'traffic', 'SO2']","{'batch_size': 100, 'epochs': 50, 'neurons': 20}",4.2,0.37,46
GRU,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'VV10']","{'batch_size': 50, 'epochs': 10, 'neurons': 500}",7.2,0.44,44
GRU,pcas,"['HR', 'NOX', 'P', 'PPT', 'RS', 'traffic']","{'batch_size': 10, 'epochs': 10, 'neurons': 50}",3.3,0.4,45
GRU,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 10, 'epochs': 10, 'neurons': 500}",11,0.47,42
LSTM,fs,"['NO', 'NO2']","{'batch_size': 10, 'epochs': 10, 'neurons': 500}",8.7,0.37,46
LSTM,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'VV10']","{'batch_size': 10, 'epochs': 10, 'neurons': 500}",11,0.43,44
LSTM,pcas,"['HR', 'NOX', 'P', 'PPT', 'RS', 'traffic']","{'batch_size': 10, 'epochs': 100, 'neurons': 100}",17,0.45,43
LSTM,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 10, 'epochs': 10, 'neurons': 50}",2.9,0.46,42
MLP,fs,"['NO', 'CO', 'traffic']","{'batch_size': 50, 'epochs': 50, 'neurons': 20}",1.8,0.41,45
MLP,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'VV10']","{'batch_size': 10, 'epochs': 10, 'neurons': 100}",2.5,0.43,44


Para esta estación, los mejores resultados los obtenemos para el modelo $MLP$. Sin embargo, como sabemos, este modelo subre de cierta inestabilidad cuando el número de capas ocultas aumenta. Sufren del denominado problema de desaparición del gradiente (*vanishing gradient*, en inglés).

Vemos, seguidamente, los resultados para la estación de **Gracia - Sant Gervasi**.

In [12]:
show_tuning_table(tuning_data, 8019044, "['NOX']", data_air_stations)

Model,vars_selection_method,x_columns,params,Duration,R2,RMSE
GRU,fs,"['NO', 'CO', 'NO2', 'O3']","{'batch_size': 10, 'epochs': 10, 'neurons': 500}",15,0.36,33
GRU,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'VV10']","{'batch_size': 50, 'epochs': 10, 'neurons': 100}",17,0.35,34
GRU,pcas,"['HR', 'NOX', 'SO2', 'T', 'traffic']","{'batch_size': 10, 'epochs': 100, 'neurons': 50}",17,0.31,35
GRU,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 50, 'epochs': 10, 'neurons': 500}",6.3,0.39,33
LSTM,fs,"['NO', 'NO2']","{'batch_size': 10, 'epochs': 10, 'neurons': 500}",12,0.33,34
LSTM,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'VV10']","{'batch_size': 50, 'epochs': 10, 'neurons': 500}",8.2,0.32,35
LSTM,pcas,"['HR', 'NOX', 'SO2', 'T', 'traffic']","{'batch_size': 10, 'epochs': 100, 'neurons': 100}",17,0.3,35
LSTM,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 10, 'epochs': 10, 'neurons': 500}",11,0.38,33
MLP,fs,"['NO', 'NO2', 'CO']","{'batch_size': 10, 'epochs': 10, 'neurons': 500}",3,0.37,33
MLP,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'VV10']","{'batch_size': 50, 'epochs': 10, 'neurons': 100}",2.5,0.33,34


Los mejores resultados los obtenemos con el modelo $GRU$ con un número alto de neuronas ocultas. En este caso, el método de selección de variables que mejores resultados ofrece es el de $FS$. Sin embargo, tal y como vemos en el blog de Peter Flom (2018) y, según expone Frank Harrell (2001) en su libro Regression Modeling Strategies, este tipo de algoritmo de selección de variables adolecen de múltiples inconvenientes. Por lo tanto, el mejor resultado lo obtenemos cuando utilizamos todas las variables independientes de nuestro conjunto de datos.

Es el turno, ahora, de analizar los resultados para la estación de **Parc Vall Hebron**.

In [13]:
show_tuning_table(tuning_data, 8019054, "['NOX']", data_air_stations)

Model,vars_selection_method,x_columns,params,Duration,R2,RMSE
GRU,fs,"['NO', 'NO2', 'SO2']","{'batch_size': 50, 'epochs': 10, 'neurons': 20}",5.7,0.32,19
GRU,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3']","{'batch_size': 50, 'epochs': 10, 'neurons': 100}",6.6,0.33,19
GRU,pcas,"['DV10', 'NOX', 'P', 'PPT', 'RS', 'SO2']","{'batch_size': 50, 'epochs': 100, 'neurons': 500}",29,0.37,18
GRU,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 10, 'epochs': 10, 'neurons': 20}",3.6,0.36,18
LSTM,fs,"['NO', 'P', 'NO2', 'traffic']","{'batch_size': 50, 'epochs': 50, 'neurons': 500}",14,0.35,19
LSTM,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3']","{'batch_size': 100, 'epochs': 100, 'neurons': 20}",7.2,0.32,19
LSTM,pcas,"['DV10', 'NOX', 'P', 'PPT', 'RS', 'SO2']","{'batch_size': 100, 'epochs': 100, 'neurons': 100}",5,0.36,18
LSTM,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 10, 'epochs': 10, 'neurons': 500}",8.8,0.35,19
MLP,fs,"['NO', 'NO2']","{'batch_size': 50, 'epochs': 50, 'neurons': 50}",2.4,0.33,19
MLP,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3']","{'batch_size': 10, 'epochs': 10, 'neurons': 50}",2.8,0.33,19


Vemos como, para esta estación, se mejora bastante en los valores de $RMSE$ obtenidos. El método de selección de variables más relevantes que mejor resultados ofrece, en este caso, es el denominado de $PCAs$, el cual selecciona la variable más relevante de cada uno de los componentes principales obtenidos del análisis $PCA$. Observamos, según este método, tienen especial relevancia, en la variabilidad de los datos, parámetros meteorológicos como: dirección del viento ( $DV10$ ), presión atmosférica ( $P$ ), precipitación ( $PPT$ ) y radiación solar ( $RS$ ).

En último lugar, vemos los resultados obtenidos para la estación de **Palau Reial**.

In [14]:
show_tuning_table(tuning_data, 8019057, "['NOX']", data_air_stations)

Model,vars_selection_method,x_columns,params,Duration,R2,RMSE
GRU,fs,"['NO', 'traffic', 'CO']","{'batch_size': 50, 'epochs': 50, 'neurons': 20}",2.9,0.45,21
GRU,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3']","{'batch_size': 10, 'epochs': 10, 'neurons': 50}",6.2,0.46,21
GRU,pcas,"['HR', 'NOX', 'P', 'RS', 'traffic']","{'batch_size': 10, 'epochs': 50, 'neurons': 20}",7.7,0.46,21
GRU,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 10, 'epochs': 10, 'neurons': 20}",2.6,0.49,20
LSTM,fs,"['NO', 'traffic', 'CO', 'NO2']","{'batch_size': 100, 'epochs': 100, 'neurons': 50}",2.9,0.46,21
LSTM,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3']","{'batch_size': 10, 'epochs': 10, 'neurons': 500}",12,0.46,21
LSTM,pcas,"['HR', 'NOX', 'P', 'RS', 'traffic']","{'batch_size': 10, 'epochs': 10, 'neurons': 100}",2.7,0.45,21
LSTM,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 100, 'epochs': 10, 'neurons': 500}",2.6,0.47,21
MLP,fs,"['NO', 'traffic']","{'batch_size': 50, 'epochs': 10, 'neurons': 100}",1.8,0.43,21
MLP,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3']","{'batch_size': 10, 'epochs': 10, 'neurons': 50}",2.4,0.45,21


Obtenemos resultados muy similares a los obtenidos para las otras estaciones. En este caso, vemos, en la selección de variables más relevantes, una mayor influencia, en la predicción para esta estación, de la variable independiente relacionada con el tránsito de vehículos. Lo cual nos hace pensar en una mayor correlación entre esta variable independiente y los niveles de óxido de nitrógeno registrados en la zona del Palacio Real.

## 2.3. Resultados para la predicción del Dióxido de Nitrógeno ( $NO_2$ )

En los modelos de predicción para el dióxido de nitrógeno ( $NO_2$) cabe esperar resultados similares a los obtenidos en el apartado anterior, para el óxido de nitrógeno ( $NO_x$ ), dada la relación existente entre estos dos gases: $NO_x = NO + NO_2$

Empezamos, como en anteriores apartados, por los resultados para la estación del **Eixample**.

In [15]:
show_tuning_table(tuning_data, 8019043, "['NO2']", data_air_stations)

Model,vars_selection_method,x_columns,params,Duration,R2,RMSE
GRU,fs,"['NOX', 'traffic', 'NO', 'CO']","{'batch_size': 100, 'epochs': 10, 'neurons': 20}",4,0.45,13
GRU,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'VV10']","{'batch_size': 10, 'epochs': 50, 'neurons': 500}",54,0.43,13
GRU,pcas,"['HR', 'NOX', 'P', 'PPT', 'RS', 'traffic']","{'batch_size': 10, 'epochs': 10, 'neurons': 100}",4.3,0.41,14
GRU,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 50, 'epochs': 50, 'neurons': 50}",3.6,0.51,12
LSTM,fs,"['NOX', 'traffic', 'NO']","{'batch_size': 50, 'epochs': 10, 'neurons': 20}",3.7,0.42,13
LSTM,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'VV10']","{'batch_size': 10, 'epochs': 10, 'neurons': 500}",11,0.42,13
LSTM,pcas,"['HR', 'NOX', 'P', 'PPT', 'RS', 'traffic']","{'batch_size': 10, 'epochs': 100, 'neurons': 50}",14,0.38,14
LSTM,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 50, 'epochs': 100, 'neurons': 20}",4.3,0.52,12
MLP,fs,"['NOX', 'SO2']","{'batch_size': 10, 'epochs': 100, 'neurons': 100}",8,0.32,15
MLP,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'VV10']","{'batch_size': 50, 'epochs': 10, 'neurons': 500}",1.9,0.46,13


Según vemos en la tabla arriba, se observa claramente un mejor resultado cuando tenemos en cuenta todas la variables independientes en los modelos. El mejor resultado lo obtenemos para la red neuronal de tipo $MLP$. Resultados similares se obtienen con las redes neuronales recurrentes: $GRU$ y $LSTM$.

Vemos ahora los resultados para otra estación de tipo tráfico urbano, la estación de **Gracia - Sant Gervasi**.

In [16]:
show_tuning_table(tuning_data, 8019044, "['NO2']", data_air_stations)

Model,vars_selection_method,x_columns,params,Duration,R2,RMSE
GRU,fs,"['NOX', 'CO', 'T', 'DV10']","{'batch_size': 10, 'epochs': 100, 'neurons': 20}",13,0.27,14
GRU,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'VV10']","{'batch_size': 10, 'epochs': 10, 'neurons': 500}",10,0.38,13
GRU,pcas,"['HR', 'NOX', 'SO2', 'T', 'traffic']","{'batch_size': 50, 'epochs': 100, 'neurons': 500}",26,0.19,15
GRU,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 50, 'epochs': 50, 'neurons': 500}",16,0.43,13
LSTM,fs,"['NOX', 'DV10', 'CO', 'SO2']","{'batch_size': 10, 'epochs': 100, 'neurons': 20}",12,0.23,15
LSTM,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'VV10']","{'batch_size': 50, 'epochs': 50, 'neurons': 500}",10,0.32,14
LSTM,pcas,"['HR', 'NOX', 'SO2', 'T', 'traffic']","{'batch_size': 50, 'epochs': 100, 'neurons': 500}",22,0.22,15
LSTM,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 10, 'epochs': 10, 'neurons': 500}",11,0.38,13
MLP,fs,"['NOX', 'CO']","{'batch_size': 50, 'epochs': 10, 'neurons': 20}",0.52,0.34,14
MLP,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'VV10']","{'batch_size': 10, 'epochs': 10, 'neurons': 500}",1.1,0.36,14


Observamos un comportamiento similar en los tres modelos basados en redes neuronales. Con una ligera mejoría en el modelo $GRU$.

Veamos los resultados para la estación de **Parc Vall Hebron**.

In [17]:
show_tuning_table(tuning_data, 8019054, "['NO2']", data_air_stations)

Model,vars_selection_method,x_columns,params,Duration,R2,RMSE
GRU,fs,"['NOX', 'P', 'T']","{'batch_size': 50, 'epochs': 50, 'neurons': 500}",16,0.3,9.4
GRU,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3']","{'batch_size': 10, 'epochs': 10, 'neurons': 500}",15,0.36,8.9
GRU,pcas,"['DV10', 'NOX', 'P', 'PPT', 'RS', 'SO2']","{'batch_size': 50, 'epochs': 50, 'neurons': 500}",14,0.36,8.9
GRU,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 10, 'epochs': 10, 'neurons': 100}",5.4,0.4,8.7
LSTM,fs,"['NOX', 'P', 'SO2']","{'batch_size': 50, 'epochs': 50, 'neurons': 500}",14,0.3,9.3
LSTM,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3']","{'batch_size': 50, 'epochs': 100, 'neurons': 500}",22,0.35,9
LSTM,pcas,"['DV10', 'NOX', 'P', 'PPT', 'RS', 'SO2']","{'batch_size': 50, 'epochs': 50, 'neurons': 500}",12,0.33,9.2
LSTM,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 10, 'epochs': 10, 'neurons': 500}",8.4,0.39,8.7
MLP,fs,"['NOX', 'traffic']","{'batch_size': 100, 'epochs': 10, 'neurons': 50}",1.9,0.31,9.2
MLP,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3']","{'batch_size': 10, 'epochs': 10, 'neurons': 500}",2.8,0.37,8.9


Podemos hablar de una mejor precisión en la predicción para el dióxido de nitrógeno, para esta estación, dado los valores tan bajos que se obtienen de $RMSE$.

Terminamos este bloque con los datos para la estación de **Palau Reial**.

In [18]:
show_tuning_table(tuning_data, 8019057, "['NO2']", data_air_stations)

Model,vars_selection_method,x_columns,params,Duration,R2,RMSE
GRU,fs,"['NOX', 'traffic']","{'batch_size': 100, 'epochs': 50, 'neurons': 20}",2.3,0.43,10
GRU,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3']","{'batch_size': 50, 'epochs': 100, 'neurons': 500}",28,0.46,9.8
GRU,pcas,"['HR', 'NOX', 'P', 'RS', 'traffic']","{'batch_size': 10, 'epochs': 50, 'neurons': 20}",7.7,0.45,9.9
GRU,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 50, 'epochs': 10, 'neurons': 50}",1.9,0.5,9.4
LSTM,fs,"['NOX', 'traffic']","{'batch_size': 100, 'epochs': 100, 'neurons': 20}",2.7,0.42,10
LSTM,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3']","{'batch_size': 50, 'epochs': 50, 'neurons': 500}",15,0.46,9.8
LSTM,pcas,"['HR', 'NOX', 'P', 'RS', 'traffic']","{'batch_size': 50, 'epochs': 100, 'neurons': 50}",4.1,0.43,10
LSTM,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 100, 'epochs': 100, 'neurons': 20}",2.9,0.49,9.5
MLP,fs,"['NOX', 'SO2', 'traffic', 'NO']","{'batch_size': 50, 'epochs': 100, 'neurons': 20}",2,0.49,9.5
MLP,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3']","{'batch_size': 50, 'epochs': 100, 'neurons': 20}",3.3,0.46,9.8


Los resultados son muy parecidos a los obtenidos para la estación anterior, de Parc Vall Hebron, con el mejor resultado obtenido para el modelo $GRU$, utilizando todas las variables independientes.

## 2.4. Resultados para la predicción del Ozono ( $O_3$ )

Terminamos este bloque con el análisis de los resultados obtenidos en la parametrización y evaluación de métodos de reducción de variables para el contaminante ozono ( $O_3$ )

El gas ozono ( $O_3$ ) tiene un efecto positivo en la estratosfera (a unos 10‐50 km de la superficie terrestre), ya que protege de la radiación ultravioleta. Sin embargo, a cotas inferiores, en la troposfera (la capa de la atmósfera en contacto con la tierra), se convierte en un contaminante que actúa como un potente y agresivo agente oxidante. Una exposición a elevados niveles del mismo origina problemas respiratorios sobre la salud humana. [Hernández & Querol, 2013]. En su formación tiene una gran influencia la elevada insolación. Por lo que en España constituye un problema generalizado.

Como siempre, empezamos con los resultados para la estación del **Eixample**.

In [19]:
show_tuning_table(tuning_data, 8019043, "['O3']", data_air_stations)

Model,vars_selection_method,x_columns,params,Duration,R2,RMSE
GRU,fs,"['NOX', 'RS', 'HR', 'VV10']","{'batch_size': 100, 'epochs': 50, 'neurons': 20}",5.3,0.48,15
GRU,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'VV10']","{'batch_size': 100, 'epochs': 50, 'neurons': 500}",12,0.69,12
GRU,pcas,"['HR', 'NOX', 'P', 'PPT', 'RS', 'traffic']","{'batch_size': 10, 'epochs': 100, 'neurons': 500}",88,0.52,15
GRU,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 10, 'epochs': 100, 'neurons': 500}",97,0.74,11
LSTM,fs,"['NOX', 'HR', 'RS']","{'batch_size': 10, 'epochs': 100, 'neurons': 500}",66,0.55,14
LSTM,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'VV10']","{'batch_size': 50, 'epochs': 50, 'neurons': 500}",14,0.68,12
LSTM,pcas,"['HR', 'NOX', 'P', 'PPT', 'RS', 'traffic']","{'batch_size': 10, 'epochs': 100, 'neurons': 500}",64,0.54,14
LSTM,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 10, 'epochs': 50, 'neurons': 100}",12,0.71,11
MLP,fs,"['NOX', 'RS', 'VV10', 'PPT']","{'batch_size': 10, 'epochs': 100, 'neurons': 50}",8.2,0.56,14
MLP,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'VV10']","{'batch_size': 100, 'epochs': 50, 'neurons': 100}",2.4,0.69,12


Los mejores resultados los obtenemos cuando consideramos todas las variables independientes. Sin embargo, cuando utilizamos el método selección de variables más relevantes $PCA0$, los resultados son muy parecidos. Como mejor modelo, el mejor resultado lo obtenemos para el modelo $GRU$.

Para la otra estación de tipo **traffic**, de **Gracia - Sant Gervasi**, obtenemos los siguiente resultados: 

In [20]:
show_tuning_table(tuning_data, 8019044, "['O3']", data_air_stations)

Model,vars_selection_method,x_columns,params,Duration,R2,RMSE
GRU,fs,"['NOX', 'RS', 'VV10', 'NO']","{'batch_size': 10, 'epochs': 100, 'neurons': 20}",13,0.44,16
GRU,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'VV10']","{'batch_size': 100, 'epochs': 50, 'neurons': 500}",8,0.64,12
GRU,pcas,"['HR', 'NOX', 'SO2', 'T', 'traffic']","{'batch_size': 10, 'epochs': 100, 'neurons': 100}",20,0.26,18
GRU,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 10, 'epochs': 50, 'neurons': 500}",53,0.66,12
LSTM,fs,"['NOX', 'RS', 'HR', 'VV10', 'PPT']","{'batch_size': 50, 'epochs': 100, 'neurons': 500}",19,0.5,15
LSTM,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'VV10']","{'batch_size': 50, 'epochs': 50, 'neurons': 500}",11,0.63,13
LSTM,pcas,"['HR', 'NOX', 'SO2', 'T', 'traffic']","{'batch_size': 10, 'epochs': 100, 'neurons': 50}",16,0.31,17
LSTM,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 10, 'epochs': 50, 'neurons': 20}",10,0.65,12
MLP,fs,"['NOX', 'VV10', 'T', 'HR', 'CO', 'RS']","{'batch_size': 10, 'epochs': 100, 'neurons': 100}",7.2,0.54,14
MLP,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'VV10']","{'batch_size': 10, 'epochs': 10, 'neurons': 20}",1.1,0.66,12


Según vemos en la tabla, los resultados entre los tres modelos basados en redes neuronales son muy similares. En este caso, el método de selección de variables $PCA0$ presenta valores de precisión muy similares a utilizar todas las variables. Sin embargo, utilizando cualquiera de los otros dos métodos de selección de variables, vemos como se produce un empeoramiento sustancial.

Pasamos ahora a analizar los resultados para las estaciones de tipo **background**, empezando por la de **Parc Vall Hebron**.

In [21]:
show_tuning_table(tuning_data, 8019054, "['O3']", data_air_stations)

Model,vars_selection_method,x_columns,params,Duration,R2,RMSE
GRU,fs,"['NO', 'T', 'RS', 'PPT']","{'batch_size': 10, 'epochs': 100, 'neurons': 20}",17,0.39,14
GRU,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3']","{'batch_size': 50, 'epochs': 50, 'neurons': 100}",8.2,0.64,11
GRU,pcas,"['DV10', 'NOX', 'P', 'PPT', 'RS', 'SO2']","{'batch_size': 50, 'epochs': 100, 'neurons': 100}",7.3,0.48,13
GRU,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 10, 'epochs': 100, 'neurons': 20}",16,0.65,11
LSTM,fs,"['NO', 'P', 'HR', 'PPT']","{'batch_size': 50, 'epochs': 100, 'neurons': 500}",23,0.2,16
LSTM,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3']","{'batch_size': 50, 'epochs': 50, 'neurons': 500}",15,0.64,11
LSTM,pcas,"['DV10', 'NOX', 'P', 'PPT', 'RS', 'SO2']","{'batch_size': 50, 'epochs': 100, 'neurons': 50}",5.9,0.5,13
LSTM,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 50, 'epochs': 100, 'neurons': 50}",6.4,0.65,11
MLP,fs,"['NO', 'RS', 'SO2', 'T']","{'batch_size': 50, 'epochs': 100, 'neurons': 500}",3.6,0.48,13
MLP,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3']","{'batch_size': 50, 'epochs': 10, 'neurons': 500}",2.3,0.65,11


Los mismos comentarios anteriores serían aplicables a los resultados obtenidos para esta estación. Quedándonos con cualquiera de los modelos basados en redes neuronales (aunque el modelo *MLP* resultaría más inestable para valores altos de neuronas en la capa oculta, como ya se ha comentado anteriormente).

Finalmente, en la tabla siguiente, vemos los resultados para la estación de **Palau Reial**.

In [22]:
show_tuning_table(tuning_data, 8019057, "['O3']", data_air_stations)

Model,vars_selection_method,x_columns,params,Duration,R2,RMSE
GRU,fs,"['NOX', 'VV10', 'RS', 'HR', 'NO2']","{'batch_size': 10, 'epochs': 50, 'neurons': 500}",47,0.46,15
GRU,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3']","{'batch_size': 10, 'epochs': 100, 'neurons': 50}",19,0.65,12
GRU,pcas,"['HR', 'NOX', 'P', 'RS', 'traffic']","{'batch_size': 10, 'epochs': 100, 'neurons': 500}",85,0.48,15
GRU,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 10, 'epochs': 50, 'neurons': 20}",7.9,0.69,12
LSTM,fs,"['NOX', 'HR', 'RS', 'VV10']","{'batch_size': 50, 'epochs': 100, 'neurons': 500}",19,0.48,15
LSTM,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3']","{'batch_size': 10, 'epochs': 100, 'neurons': 50}",17,0.65,12
LSTM,pcas,"['HR', 'NOX', 'P', 'RS', 'traffic']","{'batch_size': 10, 'epochs': 100, 'neurons': 500}",63,0.5,15
LSTM,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10', 'HR', 'P', 'PPT', 'RS', 'T', 'VV10', 'traffic']","{'batch_size': 10, 'epochs': 50, 'neurons': 50}",7.7,0.69,12
MLP,fs,"['NOX', 'T', 'NO2', 'HR', 'RS', 'P']","{'batch_size': 10, 'epochs': 50, 'neurons': 100}",4.2,0.5,15
MLP,pca0,"['CO', 'NO', 'NO2', 'NOX', 'O3']","{'batch_size': 100, 'epochs': 100, 'neurons': 50}",2.7,0.65,12


Volvemos a observar resultados similares. En todo caso, para esta estación, se observa mayor diferencia entre los modelos basados en redes neuronales y los otros modelos de RandomForest y SVM.

# Bibliografía

- Gladilin Peter and Maria Matskevichus (2019). Hyperparameters Tuning for Machine Learning Models for Time Series Forecasting. 2019 Sixth International Conference on Social Networks Analysis, Management and Security (SNAMS). 

- Orío Hernández A, Pallarés Querol M. Análisis de la calidad del aire en España. Evolución 2001-2012. Gobierno de España, Ministerio de Agricultura, Alimentación y Medio Ambiente; 2013.

- Xavier Querol. La calidad del aire en las ciudades. Un reto mundial fenosa Fg, editor.; 2018.

- Gironés J, Casas J, Minguillón J, Caihuelas R. Minería de datos - modelos y algoritmos. Primera edición ed.: Editorial UOC; 2017.

- Simone Centellegher (2020) . [How to compute PCA loadings and the loading matrix with scikit-learn](https://scentellegher.github.io/machine-learning/2020/01/27/pca-loadings-sklearn.html)

- https://stats.stackexchange.com/questions/108148/pull-out-most-important-variables-from-pca

- Wikipedia. Stepwise regression. https://en.wikipedia.org/wiki/Stepwise_regression

- Peter Flom (2018). Stopping stepwise: Why stepwise selection is bad and what you should use instead. https://towardsdatascience.com/stopping-stepwise-why-stepwise-selection-is-bad-and-what-you-should-use-instead-90818b3f52df

- Frank Harrell (2001). Regression Modeling Strategies. Chapter 5. Resampling, Validating, Describing, and Simplifying the Model. 

- Pandas documentation: [Styling](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html)
